In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                           )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.65966, saving model to best_m.h5
 - 2s - loss: 0.6875 - acc: 0.5314 - val_loss: 0.6597 - val_acc: 0.5087
Epoch 2/50
Epoch 00002: val_loss improved from 0.65966 to 0.55134, saving model to best_m.h5
 - 1s - loss: 0.6062 - acc: 0.6731 - val_loss: 0.5513 - val_acc: 0.8030
Epoch 3/50
Epoch 00003: val_loss improved from 0.55134 to 0.46053, saving model to best_m.h5
 - 1s - loss: 0.5379 - acc: 0.7456 - val_loss: 0.4605 - val_acc: 0.8304
Epoch 4/50
Epoch 00004: val_loss improved from 0.46053 to 0.36466, saving model to best_m.h5
 - 1s - loss: 0.4749 - acc: 0.7733 - val_loss: 0.3647 - val_acc: 0.8429
Epoch 5/50
Epoch 00005: val_loss improved from 0.36466 to 0.34698, saving model to best_m.h5
 - 1s - loss: 0.4160 - acc: 0.8064 - val_loss: 0.3470 - val_acc: 0.8853
Epoch 6/50
Epoch 00006: val_loss did not improve
 - 1s - loss: 0.3882 - acc: 0.8358 - val_loss: 0.3545 - val_acc: 0.8329
Epoch 7/50
Epoch 00007: val_loss improved from 0.34698 to 

Epoch 13/50
Epoch 00013: val_loss improved from 0.29215 to 0.26512, saving model to best_m.h5
 - 1s - loss: 0.2768 - acc: 0.8792 - val_loss: 0.2651 - val_acc: 0.8878
Epoch 14/50
Epoch 00014: val_loss did not improve
 - 1s - loss: 0.2783 - acc: 0.8942 - val_loss: 0.3125 - val_acc: 0.8579
Epoch 15/50
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.2751 - acc: 0.8856 - val_loss: 0.3628 - val_acc: 0.8354
Epoch 16/50
Epoch 00016: val_loss did not improve
 - 1s - loss: 0.2735 - acc: 0.8783 - val_loss: 0.2740 - val_acc: 0.8928
Epoch 17/50
Epoch 00017: val_loss improved from 0.26512 to 0.24979, saving model to best_m.h5
 - 1s - loss: 0.2809 - acc: 0.8812 - val_loss: 0.2498 - val_acc: 0.8803
Epoch 18/50
Epoch 00018: val_loss did not improve
 - 1s - loss: 0.2612 - acc: 0.8983 - val_loss: 0.3110 - val_acc: 0.8504
Epoch 19/50
Epoch 00019: val_loss did not improve
 - 1s - loss: 0.2716 - acc: 0.8864 - val_loss: 0.2528 - val_acc: 0.8878
Epoch 20/50
Epoch 00020: val_loss did not improve
 - 1s - 

Epoch 24/50
Epoch 00024: val_loss did not improve
 - 1s - loss: 0.2436 - acc: 0.8975 - val_loss: 0.2696 - val_acc: 0.8778
Epoch 25/50
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.2629 - acc: 0.8841 - val_loss: 0.2819 - val_acc: 0.8878
Epoch 26/50
Epoch 00026: val_loss improved from 0.24841 to 0.24274, saving model to best_m.h5
 - 1s - loss: 0.2655 - acc: 0.8892 - val_loss: 0.2427 - val_acc: 0.9002
Epoch 27/50
Epoch 00027: val_loss did not improve
 - 1s - loss: 0.2668 - acc: 0.8858 - val_loss: 0.2936 - val_acc: 0.8903
Epoch 28/50
Epoch 00028: val_loss did not improve
 - 1s - loss: 0.3147 - acc: 0.8639 - val_loss: 0.2508 - val_acc: 0.8978
Epoch 29/50
Epoch 00029: val_loss did not improve
 - 1s - loss: 0.2643 - acc: 0.8872 - val_loss: 0.2446 - val_acc: 0.9027
Epoch 30/50
Epoch 00030: val_loss did not improve
 - 1s - loss: 0.2600 - acc: 0.8800 - val_loss: 0.2935 - val_acc: 0.8878
Epoch 31/50
Epoch 00031: val_loss did not improve
 - 1s - loss: 0.2275 - acc: 0.9050 - val_loss: 0.246

Epoch 36/50
Epoch 00036: val_loss did not improve
 - 1s - loss: 0.2422 - acc: 0.9039 - val_loss: 0.3017 - val_acc: 0.8778
Epoch 37/50
Epoch 00037: val_loss did not improve
 - 1s - loss: 0.2388 - acc: 0.8983 - val_loss: 0.2917 - val_acc: 0.8803
Epoch 38/50
Epoch 00038: val_loss improved from 0.28968 to 0.28961, saving model to best_m.h5
 - 1s - loss: 0.2326 - acc: 0.9033 - val_loss: 0.2896 - val_acc: 0.8828
Epoch 39/50
Epoch 00039: val_loss did not improve
 - 1s - loss: 0.2168 - acc: 0.9083 - val_loss: 0.2982 - val_acc: 0.8703
Epoch 40/50
Epoch 00040: val_loss did not improve
 - 1s - loss: 0.2253 - acc: 0.9048 - val_loss: 0.2993 - val_acc: 0.8878
Epoch 41/50
Epoch 00041: val_loss did not improve
 - 1s - loss: 0.2184 - acc: 0.9092 - val_loss: 0.2914 - val_acc: 0.8828
Epoch 42/50
Epoch 00042: val_loss did not improve
 - 1s - loss: 0.2188 - acc: 0.9042 - val_loss: 0.3029 - val_acc: 0.8928
Epoch 43/50
Epoch 00043: val_loss did not improve
 - 1s - loss: 0.2464 - acc: 0.9004 - val_loss: 0.299

In [6]:
import pickle
with open('../features/cnn_2_aug2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_2_aug2_sub.csv', index=False)

0.235406598494
         id  is_iceberg
0  5941774d    0.027870
1  4023181e    0.667902
2  b20200e4    0.009352
3  e7f018bb    0.989968
4  4371c8c3    0.005837
